In [14]:
import semantic_metrics
import os
import pandas as pd
import numpy as np
import random
import importlib
importlib.reload(semantic_metrics)
from semantic_metrics import calculate_avg_similarity

 27%|██▋       | 27/100 [01:24<03:47,  3.12s/it]


In [3]:
# 加载hdx生成的QA数据
from dogeneval.utils.mongodb import load_results
qa_results = load_results(collection_name="QA_GEN-11022040")

qa_instructs = [f"{qa['Question']}\n{qa['Answer']}" for qa in qa_results]
print(len(qa_instructs))

1873


In [4]:
# 加载mmlu的指令集
mmlu_dev = "/home/junetheriver/codes/qa_generation/huawei/data/mmlu/dev"
mmlu_test = "/home/junetheriver/codes/qa_generation/huawei/data/mmlu/test"
mmlu_val = "/home/junetheriver/codes/qa_generation/huawei/data/mmlu/val"

# 读取目录下所有csv文件，将里面的题目保存在一个list中
mmlu_instructs = []
for dir in [mmlu_dev, mmlu_test, mmlu_val]:
    for file in os.listdir(dir):
        df = pd.read_csv(os.path.join(dir, file))
        # 将每行的所有列的内容拼接，df中的列没有名字
        df['merged'] = df.apply(lambda row: ''.join([str(col) for col in row]), axis=1)
        mmlu_instructs.extend(df['merged'].tolist())    

len(mmlu_instructs)

15687

In [13]:
def sample_instructs(instructs, sample_size, seed=0):
    random.seed(seed)
    return random.sample(instructs, sample_size)

sampled_qa_instructs = sample_instructs(qa_instructs, 100)
sampled_mmlu_instructs = sample_instructs(mmlu_instructs, 100)

In [6]:

avg_similarity = calculate_avg_similarity(sampled_qa_instructs)
print(avg_similarity)

2024-11-11 16:16:45.098 | INFO     | semantic_metrics:calculate_avg_similarity:21 - Calculating average similarity for 100 instructions...
100%|██████████| 100/100 [00:43<00:00,  2.31it/s]
2024-11-11 16:17:28.452 | INFO     | semantic_metrics:calculate_avg_similarity:23 - Embeddings generated for 100 instructions.
2024-11-11 16:17:28.464 | INFO     | semantic_metrics:calculate_avg_similarity:25 - Cosine similarity matrix calculated for 100 instructions.


0.5928722300696991


In [7]:
sampled_mmlu_avg_similarity = calculate_avg_similarity(sampled_mmlu_instructs)
print(sampled_mmlu_avg_similarity)


2024-11-11 16:17:38.316 | INFO     | semantic_metrics:calculate_avg_similarity:21 - Calculating average similarity for 100 instructions...
100%|██████████| 100/100 [00:18<00:00,  5.47it/s]
2024-11-11 16:17:56.598 | INFO     | semantic_metrics:calculate_avg_similarity:23 - Embeddings generated for 100 instructions.
2024-11-11 16:17:56.601 | INFO     | semantic_metrics:calculate_avg_similarity:25 - Cosine similarity matrix calculated for 100 instructions.


0.4639597389547528


In [11]:
def average_sampling(instructs, sample_size, num_folds=10):
    avg_similarities = []
    for i in range(num_folds):
        sampled_instructs = sample_instructs(instructs, sample_size, seed=i)
        avg_similarity = calculate_avg_similarity(sampled_instructs)
        avg_similarities.append(avg_similarity)
    return np.mean(avg_similarities)

In [15]:
qa_avg_similarity = average_sampling(qa_instructs, 100)
print(qa_avg_similarity)
mmlu_avg_similarity = average_sampling(mmlu_instructs, 100)
print(mmlu_avg_similarity)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


0.586265983395978
0.465850584289243
